In [1]:
from pandasql import sqldf, load_meat, load_births
import pandas as pd

In [2]:
pysqldf = lambda q: sqldf(q, globals())

In [3]:
report_views = pd.DataFrame({"orgid": ["1", "2", "1", "4"],
                   "userid": ["1", "2", "2", "3"],
                   "visitdate": ["2021-09-28", "2021-08-15", "2021-08-16", "2021-05-17"]})

org_card = pd.DataFrame({"orgid": ["1", "2", "3", "4"],
                   "status": ["Active", "Deleted", "Active", "Active"],
                   "marketcode": ["US", "UK", "AU", "US"]})


user_card = pd.DataFrame({"userid": ["1", "2", "3", "4"],
                   "practicestaff": ["1", "0", "0", "1"]})


In [4]:
q="""
    SELECT  *,
            SUM(CASE WHEN PracStaff > 0 AND NonPracStaff > 0 Then 1 ELSE 0 END) AS '#Mixed'
    FROM (
        SELECT  strftime('%m',report_views.visitdate) AS 'Month',
                SUM(CASE WHEN user_card.practicestaff = 1 THEN 1 ELSE 0 END) AS 'PracStaff',
                SUM(CASE WHEN user_card.practicestaff = 0 THEN 1 ELSE 0 END) AS 'NonPracStaff'
        FROM    report_views, org_card, user_card   
        WHERE   org_card.marketcode = "US"
            AND org_card.orgid = report_views.orgid
            AND user_card.userid = report_views.userid
            AND org_card.status ="Active"
            AND report_views.visitdate > '2020-12-31'
        GROUP BY strftime('%m',report_views.visitdate)
    ) 
"""
    


In [5]:
joined = pysqldf(q)
joined

,Month,PracStaff,NonPracStaff,#Mixed
0,05,0,1,0
